# Yvettes Notebook

In [1]:
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests
from bs4 import BeautifulSoup
from env import github_token, github_username
import re
import pandas as pd
import time

### Acquire webscrape urls from repos and save to csv

Keytakaway from urls collection:
After some exploration of urls in github search of most starred repos we noticed that the page number in the url was the only change from one page to another. Creating a loop to change page when web scraping will allow us to 
iterate through pages and get more urls into our csv file.

We also notice that the link to the repo was under <a> class = "v-align-middle". We will add a forloop to scrape each link and remove the first "/" and save into csv file.
    
In total we were able to collect 1000 observation of urls 853 of which seem to be unique.

In [2]:
    #'search?q=stars%3A%3E0&s=stars&type=Repositories'
    #'search?p=2&q=stars%3A%3E0&s=stars&type=Repositories'
    #'search?p=3&q=stars%3A%3E0&s=stars&type=Repositories'
    
#https://github.com/search?q=stars%3A%3E0&s=stars&type=Repositories
#https://github.com/search?p=100&q=stars%3A%3E0&s=stars&type=Repositories

In [3]:
#urls=[]
#for i in range(0,100):
#    url = f"https://github.com/search?p={i}&q=stars%3A%3E0&s=stars&type=Repositories"
#    reqs = requests.get(url)
#    soup = BeautifulSoup(reqs.text, 'html.parser')
#    time.sleep(2)
#    for link in soup.find_all('a',class_="v-align-middle"):
#        link = re.sub(r'/', '', link.get('href'), count = 1)
#        urls.append(link)
#        time.sleep(1)
    

In [4]:
#urls =pd.DataFrame(urls)

In [5]:
#urls.nunique()

In [6]:
#urls.to_csv('urls_final1.csv', index=False)

### acquire content of urls readme 

We will call on our saved urls_final.csv file to scrape the content of the repos of ReadMe text, main language of code. We will then save the information along with the repos name into a data frame.

In [136]:
urls_repo = pd.read_csv('urls_final.csv', index_col=0)
urls_repo

,0
0,freeCodeCamp/freeCodeCamp
1,getify/You-Dont-Know-JS
2,microsoft/vscode
3,electron/electron
4,axios/axios
...,...
995,attic-labs/noms
996,libretro/RetroArch
997,score-spec/spec
998,ThreeMammals/Ocelot


In [137]:
urls_repo.head()

,0
0,freeCodeCamp/freeCodeCamp
1,getify/You-Dont-Know-JS
2,microsoft/vscode
3,electron/electron
4,axios/axios


In [138]:
urls_repo.nunique()

0    853
dtype: int64

In [139]:
urls_repo.drop_duplicates(inplace=True)

In [140]:
urls_repo[460:470]

,0
526,bootstrap-vue/bootstrap-vue
527,sharkdp/hyperfine
528,Rigellute/spotify-tui
529,williamfiset/Algorithms
530,jina-ai/jina
531,dropzone/dropzone
532,ethereumbook/ethereumbook
533,chaozh/awesome-blockchain-cn
534,eip-work/kuboard-press
535,semantic-release/semantic-release


In [141]:
urls_repo.iloc[468]

0    eip-work/kuboard-press
Name: 534, dtype: object

In [144]:
# scrapping functions where not functionin properly because url index 468 no longer existed, so it was dropped.
#0    eip-work/kuboard-press
#Name: 534, dtype: object
urls_repo.drop([534], axis = 0, inplace= True)

KeyError: '[534] not found in axis'

In [145]:
urls_repo[460:470]

,0
526,bootstrap-vue/bootstrap-vue
527,sharkdp/hyperfine
528,Rigellute/spotify-tui
529,williamfiset/Algorithms
530,jina-ai/jina
531,dropzone/dropzone
532,ethereumbook/ethereumbook
533,chaozh/awesome-blockchain-cn
535,semantic-release/semantic-release
537,framework7io/framework7


In [148]:
urls_repo.reset_index(inplace=True)

In [149]:
urls_repo.drop(columns=['index'], inplace=True)

In [150]:
urls_repo.head()

,0
0,freeCodeCamp/freeCodeCamp
1,getify/You-Dont-Know-JS
2,microsoft/vscode
3,electron/electron
4,axios/axios


In [151]:
urls_repo.iloc[466]

0    ethereumbook/ethereumbook
Name: 466, dtype: object

In [153]:
"""
A module for obtaining repo readme and language data from the github API.
Before using this module, read through it, and follow the instructions marked
TODO.
After doing so, run it like this:
    python acquire.py
To create the `data.json` file that contains the data.
"""


# TODO: Make a github personal access token.
#     1. Go here and generate a personal access token: https://github.com/settings/tokens
#        You do _not_ need select any scopes, i.e. leave all the checkboxes unchecked
#     2. Save it in your env.py file under the variable `github_token`
# TODO: Add your github username to your env.py file under the variable `github_username`
# TODO: Add more repositories to the `REPOS` list below.

REPOS = urls_repo['0']

0-250 seem okay 
250-500  had an error
460-470 had an error
480-500 was good
Repo index number 468 eip-work/kuboard-press no longer exists and is causing problems. # 468 will be dropped from csv



In [154]:
headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )

In [155]:
def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        if "language" not in repo_info:
            raise Exception(
                "'language' key not round in response\n{}".format(json.dumps(repo_info))
            )
        return repo_info["language"]
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_download_url = get_readme_download_url(contents)
    if readme_download_url == "":
        readme_contents = ""
    else:
        readme_contents = requests.get(readme_download_url).text
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }


def scrape_github_data() -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    return [process_repo(repo) for repo in REPOS]

In [156]:
if __name__ == "__main__":
    try:
        data = scrape_github_data()
        json.dump(data, open("data.json", "w"), indent=1)
    except: 
        pass

In [157]:
# scrape repos for information.
df = scrape_github_data()

In [158]:
df = pd.DataFrame(df)

In [159]:
df.head()

,repo,language,readme_contents
0,freeCodeCamp/freeCodeCamp,TypeScript,[![freeCodeCamp Social Banner](https://s3.amaz...
1,getify/You-Dont-Know-JS,None,# You Don't Know JS Yet (book series) - 2nd Ed...
2,microsoft/vscode,TypeScript,"# Visual Studio Code - Open Source (""Code - OS..."
3,electron/electron,C++,[![Electron Logo](https://electronjs.org/image...
4,axios/axios,JavaScript,"<h1 align=""center"">\n <b>\n <a href=""..."


In [161]:
#urls.to_csv('urls_final1.csv', index=Fal
df.to_csv('readme_df.csv', index = False)

In [ ]:
df.language.isnull().sum()

In [ ]:
df.language.value_counts()

In [ ]:
df.info()

### Prepare Data

In [ ]:
def basic_clean(string):
    '''
    This function takes in a string and
    returns the string normalized.
    '''
    # we will normalize our data into standard NFKD unicode, feed it into an ascii encoding
    # decode it back into UTF-8
    string = unicodedata.normalize('NFKD', string)\
             .encode('ascii', 'ignore')\
             .decode('utf-8', 'ignore')
    # utilize our regex substitution to remove our undesirable characters, then lowercase
    string = re.sub(r"[^\w0-9'\s]", '', string).lower()
    return string

def tokenize(string):
    '''
    This function takes in a string and
    returns a tokenized string.
    '''
    # make our tokenizer, taken from nltk's ToktokTokenizer
    tokenizer = nltk.tokenize.ToktokTokenizer()
    # apply our tokenizer's tokenization to the string being input, ensure it returns a string
    string = tokenizer.tokenize(string, return_str = True)
    
    return string

def lemmatize(string):
    '''
    This function takes in string for and
    returns a string with words lemmatized.
    '''
    # create our lemmatizer object
    wnl = nltk.stem.WordNetLemmatizer()
    # use a list comprehension to lemmatize each word
    # string.split() => output a list of every token inside of the document
    lemmas = [wnl.lemmatize(word) for word in string.split()]
    # glue the lemmas back together by the strings we split on
    string = ' '.join(lemmas)
    #return the altered document
    return string

def remove_stopwords(string, extra_words = [], exclude_words = []):
    '''
    This function takes in a string, optional extra_words and exclude_words parameters
    with default empty lists and returns a string.
    '''
    # assign our stopwords from nltk into stopword_list
    stopword_list = stopwords.words('english')
    # utilizing set casting, i will remove any excluded stopwords
    stopword_list = set(stopword_list) - set(exclude_words)
    # add in any extra words to my stopwords set using a union
    stopword_list = stopword_list.union(set(extra_words))
    # split our document by spaces
    words = string.split()
    # every word in our document, as long as that word is not in our stopwords
    filtered_words = [word for word in words if word not in stopword_list]
    # glue it back together with spaces, as it was so it shall be
    string_without_stopwords = ' '.join(filtered_words)
    # return the document back
    return string_without_stopwords

In [ ]:
# apply basic clean to content
df['clean_text']= df.readme_contents.apply(basic_clean)

In [ ]:
# atokenize 
df['clean_text']= df.clean_text.apply(tokenize)

In [ ]:
# lemmatize
df['clean_text']= df.clean_text.apply(lemmatize)

In [ ]:
# remove stopwords
df['clean_text']= df.clean_text.apply(remove_stopwords)

In [ ]:
df

# Split data

In [ ]:
def split_data(df, target):
    '''
    split_date takes in a dataframe  and target variable and splits into train , validate, test 
    and stratifies on target variable
    
    The split is 20% test 80% train/validate. Then 30% of 80% validate and 70% of 80% train.
    Aproximately (train 56%, validate 24%, test 20%)
    
    returns train, validate, and test 
    '''
    # split test data from train/validate
    train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=123, 
                                        stratify=df[target])

    # split train from validate
    train, validate = train_test_split(train_validate, test_size=.3, 
                                   random_state=123, 
                                   stratify=train_validate[target])

                                   
    return train, validate, test

In [ ]:
train, validate, test = split_data(inshort_df,'')

# Explore

### look at how many of each type